In [65]:
import pandas as pd
import MeCab
from gensim import models

df = pd.read_csv("../data/tweets.csv")
mecab = MeCab.Tagger ("-Owakati")
print(mecab.parse("pythonが大好きです"))

df = df[pd.isnull(df.retweeted_status_id)]
df = df[pd.isnull(df.in_reply_to_status_id)]

for index, row in df.iterrows():
    if not pd.isnull(row.text):
        df.set_value(index, "text", mecab.parse(row["text"]))        
        
for index, row in df.iterrows():
    if not pd.isnull(row.text):
        print(row.text)
    if index > 10 : break;

python が 大好き です 

無理 だ ー 

人間 よ 、 24 に 、 分類 さ れよ ！ 

計算 中 ・ ・ ・ 

えい 、 ` from scipy . cluster . vq import vq , kmeans , whiten ` 

k - means やる ぞ 

17 件 の コメント https :// t . co / 4 wBdeSUFov “ Rails エンジニア に 役立つ Jupyter Notebook と iRuby - クック パッド 開発 者 ブログ ” https :// t . co / gn 3 VupaJH 1 

1 時間 で 配送 できる 理由 が わから ん https :// t . co / 9 q 6 R 0 jnR 4 D 

お は よー ！ 6 h 28 m ね た よ ！ January 27 , 2016 at 01 : 19 AM → January 27 , 2016 at 08 : 06 AM 

これ は 楽しみ すぎる やろ / “ パーフェクト ボディ を 目指し て ( １ 日 経過 ） - alwaysneet な 毎日 ” https :// t . co / ounxQFuw 7 W 

ねむい ー 

" Microsoft が 機械 学習 ツール キット CNTK を GitHub に 移し MIT ライセンス を 適用 " # tech # feedly https :// t . co / G 8 iqMk 5 q 44 



In [55]:
df.to_csv("../data/formatted_tweets.csv")

In [56]:
import itertools
class LabeledLineSentence(object):
    def __init__(self, df):
        self.df = df

    def __iter__(self):
        for index, row in self.df.iterrows():            
            yield models.doc2vec.LabeledSentence(row["text"].split(), ['text_%s' % index])
            

sentence = LabeledLineSentence(df)
print(list(sentence)[:5])
len(df)

[LabeledSentence(words=['無理', 'だ', 'ー'], tags=['text_1']), LabeledSentence(words=['人間', 'よ', '、', '24', 'に', '、', '分類', 'さ', 'れよ', '！'], tags=['text_2']), LabeledSentence(words=['計算', '中', '・', '・', '・'], tags=['text_3']), LabeledSentence(words=['えい', '、', '`', 'from', 'scipy', '.', 'cluster', '.', 'vq', 'import', 'vq', ',', 'kmeans', ',', 'whiten', '`'], tags=['text_4']), LabeledSentence(words=['k', '-', 'means', 'やる', 'ぞ'], tags=['text_5'])]


16435

In [57]:
from gensim import models
model = models.Doc2Vec()
model.build_vocab(sentence)

for _ in range(10):
    model.train(sentence)
    print("トレーニング" + str(_) + "終了")

トレーニング0終了
トレーニング1終了
トレーニング2終了
トレーニング3終了
トレーニング4終了
トレーニング5終了
トレーニング6終了
トレーニング7終了
トレーニング8終了
トレーニング9終了


In [26]:
# model.vocab.keys()

In [58]:
model.most_similar("リクルート")

[('-｜', 0.4989011883735657),
 ('企業', 0.48957908153533936),
 ('さくら', 0.4730980396270752),
 ('開始', 0.46794670820236206),
 ('サイバーエージェント', 0.42725899815559387),
 ('初心者', 0.41618603467941284),
 ('クリエイター', 0.4149061441421509),
 ('学生', 0.41168373823165894),
 ('女子', 0.4000186026096344),
 ('2014', 0.3906862437725067)]

In [59]:
base = "text_1"
key, index = base.split("_")
print(df.ix[int(index)][key])
for offset, similality in model.docvecs.most_similar(base):
    doctag = model.docvecs.offset2doctag[offset]
    key, index = doctag.split("_")
    print("%s%%"%int(similality*100),df.ix[int(index)][key])

無理 だ ー 

72% 無料 で 遊ん だ ー 

63% なんと ！ cRT @ yoichineji : cocos 2 d - html 5 v 2 . 0 が リリース さ れ た よう だ ー 

61% ちょ ー いい ランチ だっ た ！ たのし ー 

61% 徐々に 英語 に なっ て いく ぜ ー 

58% AppBank Games の 人 だ ー # idevjp 

58% だ ー ぱん の リバー シ だ 。 こういう キャラ 戦略 よい なぁ 。 / “ Connecting to the iTunes Store .” http :// t . co / 6 l 7 VAkr 9 Jd 

58% いい 質問 だ 

58% wercker 便利 だ 。 無料 だ 。 

58% さすが です ！ RT @ oshiro _ te : @ kasajei いい よ ね ！ いたみ お ー とめる きゃー んで ー い ー ！ 

57% タカ ノリ w RT @ RTsokuho : 見ろ よ … こいつ 今年 で 42 だ ぜ … http :// t . co / zX 7 izCsT hideattwtr 



In [60]:
from scipy.cluster.vq import vq, kmeans, whiten
centroid, destortion = kmeans(model.docvecs, 10 , iter=100, thresh=1e-05)

In [61]:
labels, dist = vq(model.docvecs, centroid)
print(labels[:5]) # どのクラスターに入ってるか
print(dist[:5]) # 中心からの距離

[8 9 4 6 2]
[ 0.47283798  0.48287922  0.25696802  0.48280719  0.36849758]


In [62]:
df = pd.read_csv("../data/tweets.csv")
for label in range(10):
    print(label,"スタート")
    print_i = 0
    for i in range(model.docvecs.count):
        if labels[i] == label:
            dist_vec = vq([model.docvecs[i]], centroid)
            doctag = model.docvecs.offset2doctag[i]
            key, index = doctag.split("_")
            if int((1.-dist[i])*100) > 70 and not pd.isnull(df.ix[int(index)][key]):
#                 print(dist_vec)
                print("%s%%"%int((1.-dist[i])*100),df.ix[int(index)][key].strip("\t\n"))
                print_i +=1
                if print_i > 3 : 
                    print("\n");break;

0 スタート
71% なにこれ
71% R&amp;Dはマネジメントできるか https://t.co/LRbM4i7oum #スマートニュース
72% すげー / “風邪 - Wikipedia” https://t.co/nQY4DaKsV6
71% おはよー！ 8h 9m ねたよ！ November 7, 2015 at 01:16AM →November 7, 2015 at 09:44AM


1 スタート
74% すごい / “ステビア - Wikipedia” https://t.co/GAOJP72XPu
71% 店舗で予約できるだろうか
73% Face swapping with Python, dlib, and OpenCV  http://t.co/bJwFe7JxYU
72% なるほど、リツイートとファボも報酬にするのか


2 スタート
72% 体をアルカリ性に変えてこ
76% 出来てきた https://t.co/AmnSJCo2bm
72% 届いたよ！ http://t.co/xBFgoTbVu5
71% やってない！ http://t.co/1yhUvSdYSe


3 スタート
71% アーロンチェアいいなぁ
75% すごい RT MACALLAN(マッカラン) の ザ・マッカラン リフレクション 700ml を Amazon でチェック！ https://t.co/u69l9wlPjk @さんから
72% 1件のコメント https://t.co/MTaTEX7rJb “ケトン体・ケトン食とは” https://t.co/ceRUkRGcwj
73% Haskel RT @lotz84_: Ocaml https://t.co/8s66iPbdej


4 スタート
74% 計算中・・・
74% ジョコビッチ https://t.co/h3xZlL9I02
72% ややこしい結果に  https://t.co/2ZnjCvyeaG https://t.co/gq7fSZZymj
84% 体重推移 https://t.co/tIGtdKgzFy


5 スタート
72% クロックスもやすい！

crocs(クロックス) の [クロックス] crocs サンダル Crocband を Amazon でチェック！ https://t